# ML App Deployment: Lung Segmentation

Earlier in the course, you created a powerful ML model capable of segmenting out the lungs from CT scans in the Lung Segmentation project! 

Now, how could we actually go ahead and deploy such a model to doctors and clinics? How might we package such a system?

One option would be to develop an ML web app! We'll go over how to do that in this notebook using HTML and Flask.

Outline:


1.   Setup ML Model
2.   Create Front-end User Interface with HTML
3.   Set up Back-end server with Flask



# Setup

Let's start off by saving the U-Net model to a file in the **Lung Segmentation notebook**. Add a code cell after the model's been trained and copy/paste the following code there to save the unet model to a file. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
save_path = F"/content/gdrive/My Drive/unet.zip" 

tf.keras.models.save_model(unet,'unet')
import zipfile

import os
import zipfile

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))


zipf = zipfile.ZipFile(save_path, 'w', zipfile.ZIP_DEFLATED)
zipdir('unet', zipf)
zipf.close()

This code will save the model to a file in Google Drive. From there, we'll re-download it when we begin writing the back-end code for our app.

#Front-end

In [ ]:
#@title Let's import our Python packages! { display-mode: "form" }
!pip install flask-ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, send_file, request, Response, jsonify
from flask import send_file

!pip install -U flask-cors
from flask_cors import CORS, cross_origin
import requests

import numpy as np
import tensorflow as tf
import cv2
from google.colab.patches import cv2_imshow

import codecs
import base64

!pip install -U jsonpickle
import jsonpickle

import gdown
import zipfile

from google.colab import drive
drive.mount('/content/gdrive')

import os
os.makedirs("static/js")

url = 'https://drive.google.com/uc?id=1J8WDgyXvOgJ3gwmJ76yxb-Q52R-ExEVD'
output = 'static/js/lung_segmentation_script.js'
gdown.download(url, output, quiet=False)

One of the biggest aspects of a web app is the ***front-end***. The front-end of a website consists of the elements that a user interacts with in the user interface (UI). Examples of front-end elements include text boxes, images, buttons, and sliders. 

What UI elements will we need to have? Are there any styles of designs that you think might work well for our app's UI?

In [ ]:
# Your Response Here

To create our Lung Segmentation app's user interface we'll be primarily using two languages: HTML and CSS. HTML is used to define UI elements (buttons, text inputs, etc.) while CSS is used to customize design attributes (color, spacing, etc.).

HTML elements are defined by `<tags>`. Each HTML element has a start `<tag>` and a closing `</tag>`, which has a backslash. Elements can also have attributes, which specify certain qualities. For example an `<img>` tag could have the attributes width and height, as seen through `<img width=100 height=100>`

Let's begin by creating a simple webpage with a few HTML elements! Two methods of displaying text to the user are below.

Paragraph Text: Text specified within the `<p>` tags are formatted to look like a block of text.

In [ ]:
<p>Paragraph Text</p>

Header Text. Text specified within the `<h>` tags are formatted to look like a section header. The number next to to the `h` in the `<h3>` tag can be a value fron 1 to 6. `<h1>` is the biggest size while `<h6>` is the smallest.

In [ ]:
<h3>Header Text</h3>

Use these two tags to create a simple HTML page by adding your code to the code cell below.

In [ ]:
%%writefile simple-webpage.html
<!-- Add your HTML elements here -->

To view our website run the code cell below and click on the URL with `ngrok.io`. If your HTML is not located in `simple-webpage.html`, then double click the code cell and edit the HTML file path.

In [ ]:
#@title Run this to host our website! { display-mode: "form" }

app = Flask(__name__)
run_with_ngrok(app)
CORS(app)

simple_webpage = "simple-webpage.html"
simple_html = codecs.open(simple_webpage, 'r').read()

@app.route("/")
def home():
    return simple_html

app.run()

Let's go over the syntax for some HTML elements. Then, you can go ahead and organize these elements however you would like! To implement a UI element you will have to copy and paste the code for the specific element in your HTML editor. You can edit the specific details such as the size or color afterwards.

Comments

In [ ]:
<!-- This is a Comment -->

Linking CSS Libraries (Different UI styles)

In [ ]:
<link href="https://fonts.googleapis.com/icon?family=Material+Icons" rel="stylesheet">
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/css/materialize.min.css">

Linking Script Libraries. These are similar to Python libraries or Node.js packages.

In [ ]:
<script src="https://cdnjs.cloudflare.com/ajax/libs/materialize/1.0.0/js/materialize.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@2.0.0/dist/tf.min.js"></script>

Title Bar

In [ ]:
<nav class="{Enter Your Color Here}">
    <div class="nav-wrapper">
      <a href="#" class="brand-logo">Skin Cancer Diagnosis</a>
    </div>
  </nav>

A UI container. For example `<div style="padding:5%;">` will have a 5% gap from the edges of the window. 

In [ ]:
<div>

</div>

Image Input

In [ ]:
<div class="file-field input-field">
    <div class="btn {Enter Your Color Here}">
      <span>Select Image</span>
      <input type="file" accept="image/*" onchange="onFileSelected(event)">
    </div>
    <div class="file-path-wrapper">
      <input class="file-path validate" type="text">
    </div>
  </div>

Image

In [ ]:
<img width={Enter Your Image Width} height={Enter Your Image Height}>

</img>

Paragraph Text

In [ ]:
<p>Paragraph Text</p>

Header Text.

In [ ]:
<h3>Header Text</h3>

Button

In [ ]:
<a class="{Enter Your Color Here} {Enter Your Button Size Here} waves-effect waves-light btn">Button Text</a>

Text Input with a Label

In [ ]:
<input value="Placeholder Text" id="text_input" type="text" class="validate">
<label class="active" for="text_input">Placeholder Text</label>

Great! Now take these HTML elements and design your interface! Be sure to replace the placeholder values with your specifics in mind! You can also check out this [link](https://materializecss.com/helpers.html) for more Materialize (the CSS library we're using) helpers. These can help with elements like aligning elements or formatting text. Whenever you want to view your updated website, press *Run* in the HTML editor!

To edit our code, we'll be using an HTML editor by w3schools. Use this [link](https://www.w3schools.com/tryit/tryit.asp?filename=tryhtml_default) to create write your HTML. Whenever you press run, you should see a preview for the application on the right side.

Remember to save your HTML code once you are done constructing your UI, so that we can use it for later activities, by pasting it into the section below.

In [ ]:
%%writefile webpage.html
<head>
  <!-- Import your Stylesheets here -->
</head>
<body>
  <!-- Add your HTML elements here -->
  <script src="static/js/lung_segmentation_script.js"></script>
  <script>
    <!-- Set your JS variables here -->
  </script>
</body>

Once we're done arranging our HTML UI elements, we also need to write some code that handles the logic of reading/sending our input image and receiving our predicted mask.

You can start off by adding a add a `<script> </script>` tag. Within these lines, you need to programmatically reference the HTML elements that display the input and predicted images with Javascript. Why might we want to do this?

In [ ]:
# Your Response Here

You can use the `document.getElementById()` function to achieve this. An example of this is below. Note that in Javascript, you need to use `let` or `var` as prefixes, while defining variables. While defining your variables make sure that the variable names are `imgtag` and `prediction_img` and also be sure to use the IDs are specific to your code.

In [ ]:
<script>

var imgtag = document.getElementById("{YOUR INPUT <img> TAG ID}")
var prediction_img = document.getElementById("{YOUR OUTPUT <img> TAG ID}")

</script>

We've also include a script that uploads the image the user selects and returns the predicted mask to the webpage. If you're interested in viewing this code (written in JS), you can view the file at the path `static/js/lung_segmentation_script.js` in the Colab filesystem.

Great job! Now, we've finished up the design and logic for the front-end of our web app!

# Back-end

Let's get started with writing the back-end code for our web app now. The back-end of our app will handle the actual lung mask prediction and communicate with the front-end UI elements to read and display the images.

Let's download our u-net model's file to our Google Colab environment. Now, we can work with our UI elements and ML models in the context of our server!

In [ ]:
unet_path = F"/content/gdrive/My Drive/unet.zip" 

with zipfile.ZipFile(unet_path, 'r') as zip_ref:
    zip_ref.extractall('')

Let's load our webpage HTML from a file into the Python environment.

In [ ]:
webpage = "webpage.html"
html = codecs.open(webpage, 'r').read()

Print the HTML contents of our webpage!

In [ ]:
# Your Code Here

Let's load up our U-net model now! For this, we'll use the keras `load_model()` function. Before we do that, can you think of any potential problems with loading up our model from a file?

In [ ]:
# Your Response Here

Since we defined a custom loss function to score our model, we'll need to define it again here. When we load the model, we'll pass the loss function as a parameter.

In [ ]:
def dice(true_mask, predicted_mask):
    true_mask, predicted_mask = true_mask.astype(bool).flatten(), predicted_mask.astype(bool).flatten()   
    return 2 * sum(true_mask & predicted_mask) / (sum(true_mask) + sum(predicted_mask))

Let's load up our model!

In [ ]:
unet = tf.keras.models.load_model('unet', custom_objects={'dice_loss' : dice, 'dice': dice})

Now, let's get started creating our Python Web Server! For this, we'll use a framework called Flask.

In [ ]:
#@title Initialize Flask Variables { display-mode: "form" }
app = Flask(__name__)
run_with_ngrok(app)

CORS(app)

"Routes" in web development specify the code you need to write to handle different website requests. For example, with Google, the route `/` or `www.google.com/` takes you to the Google home page. However, the route `/imghp` or `www.google.com/imghp` takes you to the Google Images page. 

For the route `/` on our website, define a function called` home()` that returns the website's HTML. Once you've done that, scroll to the bottom of the notebook to run your server and see your website in action! 

**Note:** Every time you want to update the code for a "route" you'll have to reinitialize the Flask variables. And re-run the code cells for all the other routes as well.

In [ ]:
@app.route("/")
# Your Code Here

Our front-end sends a request with the input image to the route `/predict_mask`, and expects the image's mask returned. The image that is sent to the back-end and then later returned to the front-end is actually a "base 64 string." This is just one type of an encoded image. 

The `/predict_mask` route's accompanying function's skeleton has been provided below.

Try implementing the following three elements of the function:
1.   Use the `cv2.cvtColor` and `cv2.resize` functions to convert `img` to a grayscale `64,64` image.
2.   Reshape img into `1,64,64,1` and save the predicted image mask into `img_pred`.
3.   Convert the masked image back into the shape `64,64` and set any pixel that isn't black to white. You can check out the `np.where` function to achieve this effect. The syntax for `np.where` is as follows: `np.where({EXPRESSION TO SELECT PIXELS},{NEW PIXEL VALUE},{IMAGE})`



In [ ]:
@app.route('/predict_mask', methods=['POST'])
@cross_origin(headers=['Access-Control-Allow-Origin'])
def predict_mask():
    try:
      print("Received Image")
      
      base64_img = base64.decodebytes(request.data)
      nparr = np.frombuffer(base64_img, np.uint8)
      img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
      
      # Your Code Here {1}
      
      cv2_imshow(img)

      print("Processed Image")
      
      # Your Code Here {2}

      print("Predicted Mask")

      # Your Code Here {3}
      
      cv2_imshow(img_pred)   

      retval, buffer = cv2.imencode('.jpg', img_pred)
      
      base64_pref = 'data:image/jpeg;base64,'
      img_pred_base64 = base64_pref + str(base64.b64encode(buffer))[2:-1]
      print("Sending Response")

      return jsonify({'img_data': img_pred_base64})
    except Exception as e:
      print(e)
      return "Invalid Request", 400

Run this code segment to start the server. Use the link with `.ngrok`, to access the webpage online. 

To debug your code on the website, you can right click any UI element and select "inspect element". A new window will pop up, from which you can select the "console" tab. Here you can view any errors or outputs of print statements from your Javascript code.

In [ ]:
app.run()